# Imports

In [44]:
import numpy as np
import sys, os, io
import six.moves.urllib as urllib
import tensorflow as tf
import cv2

from collections import defaultdict
from io import StringIO
from PIL import Image
from IPython.display import display
import pathlib

# Personnal imports
%run ../utils/object_detection_utils.ipynb
%run ../utils/data_utils.ipynb
%run ../global_variables.ipynb
%run ./detect_variables.ipynb

# Object detection API
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
utils_ops.tf = tf.compat.v1
tf.gfile = tf.io.gfile

MODEL_ROOT = os.path.join(OD_ROOT, "ws_bd2/")

# Loading model

In [33]:
# Model
model = tf.saved_model.load(os.path.join(MODEL_ROOT,"export/saved_model/"))
model = model.signatures['serving_default']
# Category index
PATH_TO_LABELS = os.path.join(MODEL_ROOT,"dataset/binary_label_map.pbtxt")
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


# Computing stats

In [46]:
def draw_bbs(source, bbs, bbs_scores=None, min_score=0, colors=None, text_scale=2):
    img = source.copy()
    for i, bb in enumerate(bbs):
        color = colors[i]
        if bbs_scores is None or bbs_scores[i]>min_score:
            ymin, xmin, ymax, xmax = bb
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), color, 4)
            if not bbs_scores is None:
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(img,  "{:.2f}".format(bbs_scores[i]), (xmin+5, ymin+int(text_scale*30)), font, text_scale*1, color, 3, cv2.LINE_AA)
    return img

In [47]:
annotation_root=os.path.join(DATASET_ROOT, VAL_FOLDER, ANNOTATION_FOLDER)
annotations_paths = return_all_files_in_folder_rec(annotation_root, ["xml"])
comp_images=True

print("Extracting groundtruth and predicted bounding boxes of "+str(len(annotations_paths))+" images(s).")
a = display(str(0)+"/"+str(len(annotations_paths)),display_id=True)
all_groundtruth_bbs = []
all_predicted_bbs = []
all_predicted_bbs_scores = []
for i, annotation_path in enumerate(annotations_paths[0:500]):
    a.update(str(i+1)+"/"+str(len(annotations_paths)))
    annotation = parse_annotation(annotation_path)
    filename = annotation["filename"].split(".")[0]
    if IMAGE_FOLDER is None:
        image_path = os.path.join(DATASET_ROOT, VAL_FOLDER, annotation["folder"], annotation["filename"])
    else:
        image_path = os.path.join(DATASET_ROOT, VAL_FOLDER, IMAGE_FOLDER, annotation["filename"])
    image = load_image(image_path, expand=True)
    height, width, channels = image.shape
    # Fetching groundtruth bounding boxes
    groundtruth_bbs = []
    groundtruth_bbs_colors = []
    color_dict = {"diatom":(255, 0, 0), "diatom_floue":(0, 255, 0), "diatom_debri":(0, 0, 255)}
    for obj_bb in annotation["objects"]:
        if obj_bb["name"] in ["diatom", "diatom_floue","diatom_debri"]:
        #if obj_bb["name"] in ["diatom"]:
            groundtruth_bbs.append([obj_bb["ymin"], obj_bb["xmin"], obj_bb["ymax"], obj_bb["xmax"]])
            groundtruth_bbs_colors.append(color_dict[obj_bb["name"]])

    # Fetching predicted bounding boxes
    prediction_result = run_inference_for_single_image(model, image)
    predicted_bbs = []
    predicted_bbs_scores = []
    predicted_bbs_colors = []
    for box, score in zip(prediction_result['detection_boxes'], prediction_result['detection_scores']):
        ymin, xmin, ymax, xmax = int(box[0]*height), int(box[1]*width), int(box[2]*height), int(box[3]*width)
        predicted_bbs.append([ymin, xmin, ymax, xmax])
        predicted_bbs_scores.append(score)
        predicted_bbs_colors.append((255, 0, 0))
    
    all_groundtruth_bbs.extend(groundtruth_bbs)
    all_predicted_bbs.extend(predicted_bbs)
    all_predicted_bbs_scores.extend(predicted_bbs_scores)
    
    # Exporting comparison images
    if comp_images:
        image_prediction = draw_bbs(image, predicted_bbs, colors=predicted_bbs_colors, bbs_scores=predicted_bbs_scores, min_score=0.9, text_scale=1)
        image_groundtruth = draw_bbs(image, groundtruth_bbs, colors=groundtruth_bbs_colors, text_scale=1)
        # Creating and saving comparison image
        comp_image = np.hstack((image_prediction, image_groundtruth))
        comp_image_path = os.path.join(OUTPUT_TMP, "images/", filename+"_comp.png")
        save_img(cv2.resize(comp_image, (0,0), fx=0.5, fy=0.5), comp_image_path, compress=5)
        #save_img(image_groundtruth, comp_image_path)
print("Finished!")

Extracting groundtruth and predicted bounding boxes of 3001 images(s).


'24/3001'

Image /mnt/nvme-storage/pfauregi/rw/tmp/images/01827_comp.png saved sucessfully!
Image /mnt/nvme-storage/pfauregi/rw/tmp/images/02272_comp.png saved sucessfully!
Image /mnt/nvme-storage/pfauregi/rw/tmp/images/02474_comp.png saved sucessfully!
Image /mnt/nvme-storage/pfauregi/rw/tmp/images/00666_comp.png saved sucessfully!
Image /mnt/nvme-storage/pfauregi/rw/tmp/images/02647_comp.png saved sucessfully!
Image /mnt/nvme-storage/pfauregi/rw/tmp/images/00950_comp.png saved sucessfully!
Image /mnt/nvme-storage/pfauregi/rw/tmp/images/02629_comp.png saved sucessfully!
Image /mnt/nvme-storage/pfauregi/rw/tmp/images/02663_comp.png saved sucessfully!
Image /mnt/nvme-storage/pfauregi/rw/tmp/images/00352_comp.png saved sucessfully!
Image /mnt/nvme-storage/pfauregi/rw/tmp/images/02339_comp.png saved sucessfully!
Image /mnt/nvme-storage/pfauregi/rw/tmp/images/00696_comp.png saved sucessfully!
Image /mnt/nvme-storage/pfauregi/rw/tmp/images/02588_comp.png saved sucessfully!
Image /mnt/nvme-storage/pfau

KeyboardInterrupt: 

# VOC Pascal Metrics

In [ ]:
libPath = "/home/pfauregi/lib/Object-Detection-Metrics/lib"
add_path(libPath)
from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes
from Evaluator import *
from utils import *


boundingboxes = BoundingBoxes()
for box in all_groundtruth_bbs:
    bb = BoundingBox(
        filename,
        "diatom",
        box[1],
        box[0],
        box[3],
        box[2],
        CoordinatesType.Absolute,
        bbType=BBType.GroundTruth,
        format=BBFormat.XYX2Y2)
    boundingboxes.addBoundingBox(bb)
for i,box in enumerate(all_predicted_bbs):
    bb = BoundingBox(
                filename,
                "diatom",
                box[1],
                box[0],
                box[3],
                box[2],
                CoordinatesType.Absolute,
                bbType=BBType.Detected,
                classConfidence=all_predicted_bbs_scores[i],
                format=BBFormat.XYX2Y2)
    boundingboxes.addBoundingBox(bb)
print("Finished formatting the bounding boxes!")
    
# Uncomment the line below to generate images based on the bounding boxes

# Create an evaluator object in order to obtain the metrics
evaluator = Evaluator()
# Plot Precision x Recall curve
evaluator.PlotPrecisionRecallCurve(
    boundingboxes,  # Object containing all bounding boxes (ground truths and detections)
    IOUThreshold=0.3,  # IOU threshold
    method=MethodAveragePrecision.EveryPointInterpolation,  # As the official matlab code
    showAP=True,  # Show Average Precision in the title of the plot
    savePath=".",
    showInterpolatedPrecision=True)  # Plot the interpolated precision curve
# Get metrics with PASCAL VOC metrics
metricsPerClass = evaluator.GetPascalVOCMetrics(
    boundingboxes,  # Object containing all bounding boxes (ground truths and detections)
    IOUThreshold=0.3,  # IOU threshold
    method=MethodAveragePrecision.EveryPointInterpolation)  # As the official matlab code
print("Average precision values per class:\n")
# Loop through classes to obtain their metrics
#print(metricsPerClass)
for mc in metricsPerClass:
    # Get metric values per each class
    c = mc['class']
    precision = mc['precision']
    recall = mc['recall']
    average_precision = mc['AP']
    ipre = mc['interpolated precision']
    irec = mc['interpolated recall']
    # Print AP per class
    print('%s: %f' % (c, average_precision))